In [ ]:
# Package Author Mark Benmuvhar
# thesis_filter-prods
# Version 2.0.0
# 10/16/2022

In [1]:
import numpy as np
import pandas as pd
from datetime import date
import re
import os

In [2]:
# find_use_pats looks for usage code flag via regex
# Intended for use in an apply statement using the Patent_Code column containing str
def find_use_pat(s):
    p=re.compile('U-\d+')
    try:
        if pd.notna(s):
            m=p.search(s)
            if m:
                return m[0]
        return np.nan
    except:
        print('error getting use code from ', s,'\n Found ',m[0])

In [3]:
def find_app_type(s):
    try:
        if pd.notna(s):
            p=re.compile(r'A|N')
            m=p.search(s)
            if m:
                return m[0]
        return np.nan
    except:
        print('error finding application type in ', s)

In [4]:
def find_app_nums(s):
    try:
        if pd.notna(s):
            p=re.compile(r'\d+')
            m=p.search(s)
            return m[0]
        else:
            return np.nan
    except:
        print('error finding application numbers in ', s)

In [5]:
def reduce_be(s):
    p=re.compile(r'A\w(?!\d)')
    if type(s) == str:
        if p.search(s):
            return 'A'
    return s

In [6]:
# Functional solution to address bioequivalence designation changes 
# in parallel with sponsor changes
# Forecast_BE summarizes whether certain values existed during 
# product lifecycle / ownership
# Preferance is given to products with no therepeutic concern (Ax)
# First, return product-specific comparisons (eg AB2) that 
# differetiate products within class.
# Next, identify any cases where a product is just AA or A
# Finally, return any products that always had therepeutic concerns (Bx)
# if nothing is found, return the default value.


def forecast_BE(app_no, be, adf, bdf):
    p1 = re.compile(r'^A\w\d')
    p2 = re.compile(r'^A\w(?!\d)')
    p3 = re.compile(r'^A')
    p4 = re.compile(r'^B\w')
    
    if pd.isna(be):
        if (adf['app_nums']==app_no).any():
            subdf = adf.loc[adf['app_nums'].isin([app_no]), 'be_red']
            if subdf.str.contains(p1, regex = True).any():
                return subdf[subdf.str.contains(p1, regex = True)].iloc[0]
            elif subdf.str.contains(p2, regex = True).any():
                return subdf[subdf.str.contains(p2, regex = True)].iloc[0]
            elif subdf.str.contains(p3, regex = True).any():
                return subdf[subdf.str.contains(p3, regex = True)].iloc[0]
        elif (bdf['app_nums']==app_no).any():
            subdf = bdf.loc[adf['app_nums'].isin([app_no]), 'be_red']
            if subdf.str.contains(p4, regex = True).any():
                return subdf[subdf.str.contains(p4, regex = True)].iloc[0]
    else:
        return be

In [8]:
# pdf_io
directory = ('OrangeData\Prods')
start=True
for filename in os.scandir(directory):
    if not filename.is_dir():
        data_in=pd.read_csv(filename, dtype =
                               {
                                'ds_name':str,
                                'route':str,
                                'dp_name':str,
                                'be':str,
                                'note':str,
                                'sponsor':str,
                                'strength':str,
                                'prod_no':int,   
                                'app_date':object,
                                'app_type':str,
                                'app_num':str
                              },
                            index_col = 0,
                            parse_dates = ['app_date']
                      )
        if start == True:
            df = data_in.copy()
            start = False
        else:
            df = pd.concat([df, data_in])    

In [10]:
# Cast Notes to reference substance (!) or reference product (+) boolean
# Reference products will lump together combined RS/RP designation (!+)
# For this project, only consider reference product case
# When a ref product gets pulled from market, fda can designate a ref. subs.
# This complicates the analysis and is outside the simple comparison of 
# time after reference prodcut we are considering.
dfw = df.copy()
ref_prod = dfw['note'].str.contains(pat = '\+', regex = True)
dfw.loc[ref_prod,'ref_prod'] = True
dfw.loc[~ref_prod,'ref_prod'] = False
dfw.drop('note', axis = 1, inplace = True)
dfw

,ds_name,route,dp_name,be,sponsor,strength,prod_no,app_date,app_type,app_nums,ref_prod
0,ABACAVIR SULFATE,SOLUTION; ORAL,ZIAGEN,NaN,GLAXOSMITHKLINE,EQ 20MG BASE/ML,1,1998-12-17,N,20978,True
1,ABACAVIR SULFATE,TABLET; ORAL,ZIAGEN,NaN,GLAXOSMITHKLINE,EQ 300MG BASE,1,1998-12-17,N,20977,True
2,ABACAVIR SULFATE; LAMIVUDINE,TABLET; ORAL,EPZICOM,NaN,SMITHKLINE BEECHAM,EQ 600MG BASE;300MG,1,2004-08-02,N,21652,True
3,ABACAVIR SULFATE; LAMIVUDINE; ZIDOVUDINE,TABLET; ORAL,TRIZIVIR,NaN,GLAXOSMITHKLINE,EQ 300MG BASE;150MG;300MG,1,2000-11-14,N,21205,True
4,ABARELIX,INJECTABLE; INTRAMUSCULAR,PLENAXIS,NaN,PRAECIS,100MG/VIAL,1,2003-11-25,N,21320,True
...,...,...,...,...,...,...,...,...,...,...,...
21151,ZONISAMIDE,CAPSULE; ORAL,ZONISAMIDE,AB,SUN PHARM INDS (IN),50MG,2,2006-03-17,A,77634,False
21152,ZONISAMIDE,CAPSULE; ORAL,ZONISAMIDE,AB,SUN PHARM INDS (IN),100MG,3,2006-03-17,A,77634,False
21153,ZONISAMIDE,CAPSULE; ORAL,ZONISAMIDE,AB,UNICHEM,25MG,1,2021-01-26,A,214492,False
21154,ZONISAMIDE,CAPSULE; ORAL,ZONISAMIDE,AB,UNICHEM,50MG,2,2021-01-26,A,214492,False


In [11]:
# Simplify the reference product codes
# Products may be assigned reference product status later in their existence.
# Or reference category may be lost if a product exits the market
# For this study, capture if a product has been assigned reference status since application.

# look for products that were updated during lifecycle
ref_prod = dfw.loc[dfw['ref_prod'],['app_nums', 'route', 'ref_prod']]    
ref_prod.drop_duplicates(keep = 'first', inplace = True)
dfw.loc[dfw['app_nums'].isin(ref_prod['app_nums']), 'ref_prod'] = True

In [13]:
# Address updated application types

anda = dfw[dfw['app_type']=='A'][['app_nums', 'app_type']]
nda = dfw[dfw['app_type']=='N'][['app_nums']]

to_update = nda.merge(right = anda, how = 'inner', on = ['app_nums'])
to_update.drop_duplicates(keep = 'first', inplace = True)

dfw.loc[dfw['app_nums'].isin(to_update['app_nums']), 'app_type'] = 'A'
dfw.reset_index(drop = True, inplace = True)

In [15]:
# Refactored V0_4_2
# Can classify just on app_nums and route
# https://pandas.pydata.org/pandas-docs/stable/reference/api/
#                           pandas.DataFrame.itertuples.html#pandas.DataFrame.itertuples       
# Simplifying BE classifications
# First, create df for "A*" or "B*" classifications.  These are major subtypes.
# Since route of adminsitration is preserved, second letter isn't germaine.
# Next, look for instances where a drug had an NA classification and later received an A class.
# It's also possible for drugs to later recieve a B class.
# For this project, we only need to know when a product was A class.
# This will allow determination if product was a strict generic at launch.  
# Once this is done, call forecast_BE and assign limiting case for BE class.

#anti_join syntax:
#https://www.statology.org/pandas-anti-join.  Accessed 9/8/2022


dfw['be_red']=dfw['be'].apply(reduce_be)

no_be = dfw.loc[pd.isna(dfw['be_red']), 
                    ['app_nums','ds_name', 'route', 'be_red']].drop_duplicates(keep = 'first')

abx_be = dfw.loc[dfw['be_red'].str.contains(r'^A\w\d', na = False, regex = True), 
                   ['app_nums','ds_name', 'route', 'be_red']
                ] .drop_duplicates(subset = ['app_nums','ds_name', 'route'], keep = 'last')       
a_be = dfw.loc[dfw['be_red'].str.contains(r'^A(?!\w+\d)', na = False, regex = True), 
                   ['app_nums','ds_name', 'route', 'be_red']
              ].drop_duplicates(keep = 'first')
b_be = dfw.loc[dfw['be_red'].str.contains(r'^B\D*', na = False, regex = True), 
                   ['app_nums', 'ds_name','route','be_red']
              ].drop_duplicates(keep = 'first')
              
dfw2=dfw.copy()


prod_keys = ['app_nums', 'ds_name','route']               
subdf_abx = abx_be.merge(right = dfw2[prod_keys], 
                         how = 'inner', 
                         on = prod_keys)
subdf_abx.drop_duplicates(keep = 'first', inplace = True)

a_not_abx = a_be.merge(right = abx_be[prod_keys], 
                       how = 'outer', 
                       on = prod_keys, 
                       indicator = True)
a_not_abx = a_not_abx[(a_not_abx['_merge'] == 'left_only')].drop('_merge', axis = 1)
a_not_abx.drop_duplicates(keep = 'first', inplace = True)

b_only = b_be.merge(right = abx_be[prod_keys], 
                    how = 'outer', 
                    on = prod_keys, 
                    indicator = True)
b_only = b_only[(b_only['_merge'] == 'left_only')].drop('_merge', axis = 1)
b_only = b_only.merge(right = a_not_abx[prod_keys], 
                      how = 'outer', 
                      on = prod_keys, 
                      indicator = True)
b_only = b_only[(b_only['_merge'] == 'left_only')].drop('_merge', axis = 1)
b_only.drop_duplicates(keep = 'first', inplace = True)

na_only = no_be.merge(right = abx_be[prod_keys], 
                      how = 'outer', 
                      on = prod_keys, 
                      indicator = True)
na_only = na_only[(na_only['_merge'] == 'left_only')].drop('_merge', axis = 1)
na_only = na_only.merge(right = a_not_abx[prod_keys], 
                        how = 'outer', 
                        on = prod_keys, 
                        indicator = True)
na_only = na_only[(na_only['_merge'] == 'left_only')].drop('_merge', axis = 1)
na_only = na_only.merge(right = b_only[prod_keys], 
                        how = 'outer', 
                        on = prod_keys, 
                        indicator = True)
na_only = na_only[(na_only['_merge'] == 'left_only')].drop('_merge', axis = 1)
na_only.drop_duplicates(keep = 'first', inplace = True)

dfw2.drop(['be', 'be_red'], axis = 1, inplace = True)
dfw2.drop_duplicates(keep = 'first', inplace = True)

subdfw2 = pd.concat([subdf_abx, a_not_abx, b_only, na_only])
subdfw2.rename(columns={'be_red':'be_fcst'}, inplace = True)
dfw2 = dfw2.merge(right = subdfw2, 
                  how = 'inner', 
                  on = prod_keys)

,ds_name,route,dp_name,sponsor,strength,prod_no,app_date,app_type,app_nums,ref_prod,be_fcst
0,ABACAVIR SULFATE,SOLUTION; ORAL,ZIAGEN,GLAXOSMITHKLINE,EQ 20MG BASE/ML,1,1998-12-17,N,20978,True,A
1,ABACAVIR SULFATE,SOLUTION; ORAL,ZIAGEN,VIIV HLTHCARE,EQ 20MG BASE/ML,1,1998-12-17,N,20978,True,A
2,ABACAVIR SULFATE,TABLET; ORAL,ZIAGEN,GLAXOSMITHKLINE,EQ 300MG BASE,1,1998-12-17,N,20977,True,A
3,ABACAVIR SULFATE,TABLET; ORAL,ZIAGEN,VIIV HLTHCARE,EQ 300MG BASE,1,1998-12-17,N,20977,True,A
4,ABACAVIR SULFATE; LAMIVUDINE,TABLET; ORAL,EPZICOM,SMITHKLINE BEECHAM,EQ 600MG BASE;300MG,1,2004-08-02,N,21652,True,A
...,...,...,...,...,...,...,...,...,...,...,...
55212,ZOLMITRIPTAN,SPRAY; NASAL,ZOLMITRIPTAN,PADAGIS ISRAEL,2.5MG/SPRAY,1,2021-09-30,A,212469,False,A
55213,ZOLMITRIPTAN,SPRAY; NASAL,ZOLMITRIPTAN,PADAGIS ISRAEL,5MG/SPRAY,2,2021-09-30,A,212469,False,A
55214,ZONISAMIDE,CAPSULE; ORAL,ZONISAMIDE,UNICHEM,25MG,1,2021-01-26,A,214492,False,A
55215,ZONISAMIDE,CAPSULE; ORAL,ZONISAMIDE,UNICHEM,50MG,2,2021-01-26,A,214492,False,A


In [19]:
#cleanup spacing around commas and semicolons and slashes
dfw2['route'] = dfw2['route'].str.replace(pat = ';\w', repl ='; ', regex = True)
dfw2['ds_name'] = dfw2['ds_name'].str.replace(pat = ';(?=\w)', repl ='; ', regex = True)
dfw2['ds_name'] = dfw2['ds_name'].str.replace(pat = ',(?=\w)', repl =', ', regex = True)
dfw2['dp_name'] = dfw2['dp_name'].str.replace(pat = ';(?=\w)', repl ='; ', regex = True)
dfw2['dp_name'] = dfw2['dp_name'].str.replace(pat = ',(?=\w)', repl =', ', regex = True)
dfw2['strength'] = dfw2['strength'].str.replace(pat = ' /', repl ='/', regex = True)

In [97]:
dfw2 = dfw2.drop_duplicates(keep = 'first', 
                            subset = dfw2.columns[~dfw2.columns.isin(
                                ['sponsor','app_date', 'dp_name', 'route', 'strength'])])
dfw2.reset_index(inplace = True, drop = True)

In [99]:
dfw2.to_csv('OrangeData\\for_analysis\ob_prods_full_v0_2_0.csv')

In [100]:
dfw2

,ds_name,route,dp_name,be,note,sponsor,strength,prod_no,app_date,app_type,app_nums
25,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"ACETAMINOPHEN, BUTALBITAL, AND CAFFEINE",AB,+,MIKART,325MG;50MG;40MG,1,1986-03-17,N,89007
25,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"ACETAMINOPHEN, BUTALBITAL, AND CAFFEINE",AB,+,MIKART,325MG;50MG;40MG,1,1986-03-17,N,89007
23,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"ACETAMINOPHEN, BUTALBITAL, AND CAFFEINE",AB,+,MIKART,325MG;50MG;40MG,1,1986-03-17,N,89007
21,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"ACETAMINOPHEN, BUTALBITAL, AND CAFFEINE",AB,+,MIKART,325MG;50MG;40MG,1,1986-03-17,N,89007
29,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"BUTALBITAL, ACETAMINOPHEN AND CAFFEINE",NaN,+,MIKART,325MG;50MG;40MG,1,1986-03-17,N,89007
32,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"BUTALBITAL, ACETAMINOPHEN AND CAFFEINE",NaN,+,MIKART,325MG;50MG;40MG,1,1986-03-17,A,89007
35,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"BUTALBITAL, ACETAMINOPHEN AND CAFFEINE",NaN,+,MIKART,325MG;50MG;40MG,1,1986-03-17,A,89007
40,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"BUTALBITAL, ACETAMINOPHEN AND CAFFEINE",NaN,+,MIKART,325MG;50MG;40MG,1,1986-03-17,A,89007
45,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"BUTALBITAL, ACETAMINOPHEN AND CAFFEINE",NaN,+,MIKART,325MG;50MG;40MG,1,1986-03-17,A,89007
50,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,CAPSULE; ORAL,"BUTALBITAL, ACETAMINOPHEN AND CAFFEINE",NaN,+,MIKART,325MG;50MG;40MG,1,1986-03-17,A,89007
